## Imports

In [91]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from getpass import getpass
import sys, os
import pandas as pd
from datetime import date, datetime, timedelta
import os
import pandas as pd
from datetime import date, datetime

## Set influx connection

In [92]:
# List installed package versions
packList = ["pandas", "altair", "numpy", "itkdb"]
for p in packList:
    try:
        mod = __import__(p)
        print(mod.__name__ + " : " + mod.__version__)
    except ImportError:
        print(p + " not installed")

# Get the current working directory
thisDir = os.getcwd()

# List files in the parent directory
parentDir = os.path.abspath(os.path.join(thisDir, os.pardir))
files_in_parent = os.listdir(parentDir)
print("Files in parent directory:", files_in_parent)

pandas : 1.4.4
altair : 5.0.1
numpy : 1.21.5
itkdb not installed
Files in parent directory: ['.git', '.gitignore', '.ipynb_checkpoints', 'notebooks', 'output_file.csv', 'picolog_folder', 'README.md', 'requirements.txt']


In [93]:
# Set original credentials
bucket_remote = "GLADD"
org_remote = "PPE"
token_remote = "EO3dz3f5Bee1T45loRbpyk0SmHAkjOyB0qgKsv2Zc_NPW1_IBwy-odhRgVpPxBftVre63C4eS7V55EhCWbKqHQ=="
url_remote = "194.36.1.20:8086"

print("Connection details:")
print(f"bucket_remote: {bucket_remote}")
print(f"org_remote: {org_remote}")
print(f"token_remote: {token_remote}")
print(f"url_remote: {url_remote}")



Connection details:
bucket_remote: GLADD
org_remote: PPE
token_remote: EO3dz3f5Bee1T45loRbpyk0SmHAkjOyB0qgKsv2Zc_NPW1_IBwy-odhRgVpPxBftVre63C4eS7V55EhCWbKqHQ==
url_remote: 194.36.1.20:8086


In [99]:
def GetInfluxCredentials():
    print("Getting Influx info.")
    return {
        'bucket': "GLADD",
        'org': "PPE",
        'token': "EO3dz3f5Bee1T45loRbpyk0SmHAkjOyB0qgKsv2Zc_NPW1_IBwy-odhRgVpPxBftVre63C4eS7V55EhCWbKqHQ==",
        'url': "194.36.1.20",
        'port': "8086"
    }

## Query Data

The code below takes in the flux data as well as the data from the picolog folder to perform calculations on them and produce an output in the csv file 'output_file'

In [100]:
# Timestamp converter function
def TimeStampConverter(inStr, inPat):
    timeObj = None
    if inStr == "now":
        timeObj = datetime.now()
    else:
        timeObj = datetime.strptime(inStr, inPat)
    return timeObj.strftime("%Y-%m-%dT%H:%M:%SZ")

In [101]:
### geting temp., humid. data from influx
def fetch_temperature_and_humidity(start_time_str, bucket_remote, url_remote, token_remote, org_remote):
    clientV2_remote = influxdb_client.InfluxDBClient(url=url_remote, token=token_remote, org=org_remote)
    query_api_remote = clientV2_remote.query_api()

    start_time = datetime.strptime(str(start_time_str), '%Y-%m-%d %H:%M:%S')

    filters = {'_measurement': "data"}
    query_str = f'from(bucket: \"{bucket_remote}\") |> range(start: {start_time.strftime("%Y-%m-%dT%H:%M:%SZ")})'
    for k, v in filters.items():
        query_str += f' |> filter(fn: (r) => r["{k}"] == "{v}")'
    query_str += ' |> yield(name: "mean")'

    query_result_remote = query_api_remote.query_data_frame(org=org_remote, query=query_str)

    temperature_results = []
    humidity_results = []
    for index, row in query_result_remote.iterrows():
        if row['_field'] == 'temperature':
            temperature_results.append(row.to_dict())
        elif row['_field'] == 'humidity':
            humidity_results.append(row.to_dict())

    if len(temperature_results) > 0:
        temperature = temperature_results[0]['_value']
    else:
        temperature = None

    if len(humidity_results) > 0:
        humidity = humidity_results[0]['_value']
    else:
        humidity = None

    return temperature, humidity


In [102]:
# Perform calculations function

### useful details

def perform_calculations(input_folder, output_file, credentials, timestamp_str):
    # Extract credentials
    bucket_remote = credentials['bucket']
    org_remote = credentials['org']
    token_remote = credentials['token']
    url_remote = credentials['url']
    port = credentials['port']

    all_data = []

    timestamp = TimeStampConverter(timestamp_str, "%Y-%m-%d %H:%M:%S")


    
     # Initialize the InfluxDB client with the extracted host and port
    clientV2_remote = influxdb_client.InfluxDBClient(url=url_remote, port=int(port), token=token_remote, org=org_remote)

    
    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            data = pd.read_csv(file_path)
            # Calculate Vin Drop and GND Drop
            data['Vin Drop (V)'] = data['Vin+ Last (V)'] - data['Vin- Last (V)']
            data['GND Drop (V)'] = data['GND+ Last (V)'] - data['GND- Last (V)']
            
            # Calculate Resistance Vin(Ohms) and Total Resistance(mOhms)
            data['Resistance Vin(Ohms)'] = data['Vin Drop (V)'] / 10 / 5
            data['Resistance GND(Ohms)'] = data['GND Drop (V)'] / 10 / 5
            data['Total Resistance(mOhms)'] = (data['Resistance Vin(Ohms)'] + data['Resistance GND(Ohms)']) * 1000
            
            # Calculate Capacitor Equivalent leakage current(nA)
            data['Capacitor Equivalent leakage current(nA)'] = ((data['Capacitor leakage test Last (V)'] / 10) / (1 * 10 ** 6)) / 1000000
            
            # Calculate NTC value(Kohms)
            data['NTC value (Kohms)'] = 0.2 * 51 / data['NTC Last (V)']
            
            # Fetch temperature and humidity
            temperature, humidity = fetch_temperature_and_humidity(timestamp_str, bucket_remote, url_remote, token_remote, org_remote)
            
            # Prepare the new DataFrame with desired columns and values
            result_data = pd.DataFrame({
                'component': filename,
                'componentType': 'PCB',
                'stage': 'PCB_QC',
                'testType': 'HV_LV_TEST',
                'institution': 'Glasgow',
                'runNumber': data.shape[0],
                'date': date.today().strftime('%Y-%m-%d'),
                'passed': 'YES',
                'problems': 'False',
                'property1_value': 'B.masic',
                'property1_key': 'OPERATOR',
                'property2_value': 'INSTRUMENT',
                'property2_key': '',
                'property3_value': 'ANALYSIS_VERSION',
                'property3_key': '',
                'result1_key': 'VIN_DROP',
                'result1_value': data['Vin Drop (V)'],
                'result2_key': 'GND_DROP',
                'result2_value': data['GND Drop (V)'],
                'result3_key': 'EFFECTIVE RESISTANCE',
                'result3_value': data['Total Resistance(mOhms)'],
                'result4_key': 'HV_LEAKAGE',
                'result4_value': data['Capacitor leakage test Last (V)'],
                'result5_key': 'LEAKAGE_CURRENT (nA)',
                'result5_value': data['Capacitor Equivalent leakage current(nA)'],
                'result6_key': 'NTC_VOLTAGE',
                'result6_value': data['NTC Last (V)'],
                'result7_key': 'NTC_VALUE',
                'result7_value': data['NTC value (Kohms)'],
                'result8_key': 'TERMPERATURE',
                'result8_value': temperature,
                'result9_key': 'HUMIDITY',
                'result9_value': humidity,
                'timestamp': [timestamp]  # Add the timestamp column to the DataFrame
            })
            
            all_data.append(result_data)
    
    if all_data:
        final_result = pd.concat(all_data, ignore_index=True)
        final_result.to_csv(output_file, index=False)
        print(f"Calculation completed. Output saved to {output_file}.")


In [104]:
# Run calculations
if __name__ == "__main__":
    input_folder = os.path.join(parentDir, "picolog_folder")
    output_file = os.path.join(parentDir, "output_file.csv")
    timestamp_str = "2023-08-08 14:30:00"
    
    influx_credentials = GetInfluxCredentials()
    
    perform_calculations(input_folder, output_file, influx_credentials, timestamp_str)

Getting Influx info.


C:\Users\User\anaconda3\New folder\lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "GLADD") |> range(start: 2023-08-08T14:30:00Z) |> filter(fn: (r) => r["_measurement"] == "data") |> yield(name: "mean") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


ApiException: (503)
Reason: Service Unavailable
HTTP response headers: HTTPHeaderDict({'Server': 'squid/4.15', 'Mime-Version': '1.0', 'Date': 'Wed, 09 Aug 2023 13:19:32 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Content-Length': '3984', 'X-Squid-Error': 'ERR_CONNECT_FAIL 111', 'Vary': 'Accept-Language', 'Content-Language': 'en', 'X-Cache': 'MISS from sand, MISS from localhost', 'X-Cache-Lookup': 'MISS from sand:8080, MISS from localhost:3128', 'Via': '1.1 sand (squid/4.15), 1.0 localhost (squid/3.1.19)', 'Connection': 'keep-alive'})
HTTP response body: b'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">\n<html><head>\n<meta type="copyright" content="Copyright (C) 1996-2021 The Squid Software Foundation and contributors">\n<meta http-equiv="Content-Type" CONTENT="text/html; charset=utf-8">\n<title>ERROR: The requested URL could not be retrieved</title>\n<style type="text/css"><!-- \n /*\n * Copyright (C) 1996-2021 The Squid Software Founda